# Homework 4

In [35]:
import numpy as np
import mujoco
import mujoco.viewer as viewer

xml="""<mujoco model="3R_spatial">
    <compiler angle="radian"/>
    <asset>  
        <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
        rgb2=".2 .3 .4" width="300" height="300" mark="none"/>
        <material name="grid" texture="grid" texrepeat="6 6" texuniform="true" reflectance=".2"/>
    </asset>
    <default>
        <joint type="hinge" axis="0 0 1" limited="true"/>

        <!-- thin long cylinder and fixed cylinder -->
        <geom type="cylinder" size=".025 .2" pos="0 0 0"/>
    </default>

    <worldbody>
        <light diffuse=".5 .5 .5" pos="0 0 3" dir="0 0 -1"/>
        <geom type="plane" size="1 1 0.1" material="grid"/>  
        <body name="BaseLink" pos="0 0 .3">
            <joint name="joint1" pos="0 0 0" range="-180 180" axis ="0 0 1" damping="0.1"/>
            <geom pos="0 0 0" rgba=".6 .2 .2 1" size=".0125 .3"/>
            <geom pos="0 0 0" rgba=".6 .2 .2 1" size=".025 .1"/>
            <body name="fix1" pos="0.1 0 0" euler="0 90 0">
            <geom size=".0125 .1" rgba=".2 .6 1 1"/>
                <body name="link1" pos="0 0 0.1" euler="90 0 0">
                    <joint name="joint2" pos="0 0 0" range="-90 90" axis ="0 0 1" damping="0.1"/>
                    <geom pos="0 0 0" rgba=".6 .2 .2 1" size=".0125 .2"/>
                    <geom pos="0 0 0" rgba=".6 .2 .2 1" size=".025 .1"/>
                    <body name="fix2" pos="0.1 0 0" euler="0 90 0">
                        <geom size=".0125 .1" rgba=".2 .6 1 1"/>
                        <body name="link2" pos="0 0 0.1" euler="90 0 0">
                            <joint name="joint3" pos="0 0 0" range="-90 90" axis ="0 0 1" damping="0.1"/>
                            <geom rgba=".6 .2 .2 1" size=".025 .05"/>
                            <geom rgba=".6 .2 .2 1" size=".0125 .1"/>
                            <!-- 添加 site 元素，标记末端执行器的位置 -->
                            <site name="end_effector" pos="0 0 0" size="0.001" rgba="0 1 0 1"/>
                        </body>
                    </body>
                </body>
            </body>
        </body>
    </worldbody>

    <!-- Actuators for controlling the joints -->
    <actuator>
        <position name="position_control1" joint="joint1" kp=".2"  ctrlrange="-90 90"/>
        <position name="position_control2" joint="joint2" kp="2" kv="0.02" ctrlrange="-90 90"/>
        <position name="position_control3" joint="joint3" kp="2" kv="0.02" ctrlrange="-90 90"/>
    </actuator>
    <!-- Sensors for monitoring joint positions and torques -->
    <sensor>
        <jointpos name="joint1_position_sensor" joint="joint1"/> <!-- Position sensor for joint1 -->
        <jointpos name="joint2_position_sensor" joint="joint2"/> <!-- Position sensor for joint2 -->
        <jointpos name="joint3_position_sensor" joint="joint3"/> <!-- Position sensor for joint3 -->
    </sensor>
    <keyframe>
        <!-- Initial configuration -->
        <key name="initial_pose" qpos="0 0 0"/>        
        <!-- Bent configuration -->
        <key name="bent_pose" qpos="0 0 0"/>
    </keyframe>

</mujoco>"""


In [36]:
# load
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
mujoco.mj_resetDataKeyframe(model, data, 0)
viewer.launch(model,data)

# 获取 BaseLink 的位置和旋转矩阵
try:
    base_link_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, "BaseLink")
    base_position = data.xpos[base_link_id]
    base_rotation_matrix = data.xmat[base_link_id].reshape(3, 3)
    print("BaseLink 位置（position）:", base_position)
    print("BaseLink 旋转矩阵（rotation_matrix）:\n", base_rotation_matrix)
except Exception as e:
    print("Error accessing BaseLink:", e)

# 获取 end_effector 的位置和旋转矩阵
try:
    site_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, "end_effector")
    position = data.site_xpos[site_id]
    rotation_matrix = data.site_xmat[site_id].reshape(3, 3)
    print("\n末端执行器位置（position）:", position)
    print("末端执行器旋转矩阵（rotation_matrix）:\n", rotation_matrix)
except Exception as e:
    print("Error accessing end_effector:", e)


import modern_robotics as mr
import numpy as np
M = np.array([[0, 0, 1, 0.1],
            [0, 1, 0, 0],
            [-1, 0, 1, -0.1],
            [0, 0, 0, 1]])
Blist = np.array([[0, 0, 1, 0, 0, 0],
                [0, -1, 0, 0, 0, -0.1],
                [1, 0, 0, 0, -0.1, 0]]).T
thetalist = data.qpos[:3]

print("\ntheta:", thetalist)
print("\nkinematic: ")
print(mr.FKinBody(M, Blist, thetalist))

BaseLink 位置（position）: [0.  0.  0.3]
BaseLink 旋转矩阵（rotation_matrix）:
 [[ 0.99016462  0.1399072   0.        ]
 [-0.1399072   0.99016462 -0.        ]
 [-0.          0.          1.        ]]

末端执行器位置（position）: [0.10603583 0.03908155 0.09106833]
末端执行器旋转矩阵（rotation_matrix）:
 [[ 0.71402819 -0.52504948 -0.46312718]
 [ 0.67466997  0.69274286  0.25480927]
 [ 0.18704057 -0.494399    0.84887305]]

theta: [-0.14036335 -0.15945318  0.00094514]

kinematic: 
[[-1.58778347e-01  9.33146513e-04  9.87313813e-01  1.15877791e-01]
 [-1.38128127e-01  9.90143791e-01 -2.31493822e-02 -2.71060376e-04]
 [-1.13638259e+00 -1.39118281e-01  8.30229313e-01 -8.53650781e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [ ]:
# 获取 BaseLink 的位置和旋转矩阵
try:
    base_link_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, "link1")
    base_position = data.xpos[base_link_id]
    base_rotation_matrix = data.xmat[base_link_id].reshape(3, 3)
    print("Link1 位置（position）:", base_position)
    print("Link1 旋转矩阵（rotation_matrix）:\n", base_rotation_matrix)
except Exception as e:
    print("Error accessing BaseLink:", e)

# 获取 BaseLink 的位置和旋转矩阵
try:
    base_link_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, "link2")
    base_position = data.xpos[base_link_id]
    base_rotation_matrix = data.xmat[base_link_id].reshape(3, 3)
    print("Link2 位置（position）:", base_position)
    print("Link2 旋转矩阵（rotation_matrix）:\n", base_rotation_matrix,'\n')
except Exception as e:
    print("Error accessing BaseLink:", e,'\n')

# 获取 BaseLink 的位置和旋转矩阵
try:
    base_link_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, "link3")
    base_position = data.xpos[base_link_id]
    base_rotation_matrix = data.xmat[base_link_id].reshape(3, 3)
    print("Link3 位置（position）:", base_position)
    print("Link3 旋转矩阵（rotation_matrix）:\n", base_rotation_matrix)
except Exception as e:
    print("Error accessing BaseLink:", e)

# # 获取 end_effector 的位置和旋转矩阵
# try:
#     site_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, "end_effector")
#     position = data.site_xpos[site_id]
#     rotation_matrix = data.site_xmat[site_id].reshape(3, 3)
#     print("\n末端执行器位置（position）:", position)
#     print("末端执行器旋转矩阵（rotation_matrix）:\n", rotation_matrix)
# except Exception as e:
#     print("Error accessing end_effector:", e,'\n')


import modern_robotics as mr
import numpy as np
M = np.array([[0, 0, 1, 0.2],
            [0, 1, 0, 0],
            [-1, 0, 0, 0.1],
            [0, 0, 0, 1]])
Blist = np.array([[0, 0, 1, 0, 0, 0],
                [0, -1, 0, 0.3, 0, -0.2],
                [1, 0, 0, 0, 0.1, 0]]).T
thetalist = data.qpos[:3]

print("\ntheta:", thetalist)
print("\nkinematic: ")
print(mr.FKinBody(M, Blist, thetalist))

## v0.2

In [2]:
import numpy as np
import mujoco
import mujoco.viewer as viewer

import numpy as np
import mujoco
import mujoco.viewer as viewer

xml="""<mujoco model="3R_spatial">
    <option gravity="0 0 0"/>
    <compiler angle="degree"/>
    <asset>  
        <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
                 rgb2=".2 .3 .4" width="300" height="300" mark="none"/>
        <material name="grid" texture="grid" texrepeat="6 6" texuniform="true" reflectance=".2"/>
    </asset>
    <default>
        <joint type="hinge" axis="0 0 1" limited="true"/>
        <geom type="cylinder" size=".025 .2" pos="0 0 0"/>
    </default>

    <worldbody>
        <light diffuse=".5 .5 .5" pos="0 0 3" dir="0 0 -1"/>
        <geom type="plane" size="1 1 0.1" material="grid"/>  
        <body name="link1" pos="0 0 .3">
            <joint name="joint1" pos="0 0 0" range="-180 180" axis="0 0 1" damping="0.1"/>
            <geom pos="0 0 0" rgba=".6 .2 .2 1" size=".0125 .3"/>
            <geom pos="0 0 0" rgba=".6 .2 .2 1" size=".025 .1"/>
            <geom name="fix1" pos=".1 0 0" euler="0 90 0" size=".0125 .1" rgba=".2 .6 1 1"/>
            <body name="link2" pos=".2 0 0" euler="90 0 -90">
                <joint name="joint2" pos="0 0 0" range="0 180" axis="0 0 1" damping="0.1"/>
                <geom pos="0 0 0" rgba=".6 .2 .2 1" size=".0125 .3"/>
                <geom pos="0 0 0" rgba=".6 .2 .2 1" size=".025 .1"/>
                <geom name="fix2" pos=".1 0 0" euler="0 90 0" size=".0125 .1" rgba=".2 .6 1 1"/>
                <body name="link3" pos=".2 0 0" euler="-90 0 0">
                    <joint name="joint3" pos="0 0 0" range="-90 90" axis="0 0 1" damping="0.1"/>
                    <geom pos="0 0 0" rgba=".6 .2 .2 1" size=".0125 .15"/>
                    <geom pos="0 0 0" rgba=".6 .2 .2 1" size=".025 .05"/>
                    <site name="end_effector" pos="0 0 0" size="0.001" rgba="0 1 0 1"/>
                </body>
            </body>
        </body>
    </worldbody>


    <actuator>
        <position name="position_control1" joint="joint1" kp=".2" ctrlrange="-3.1416 3.1416"/>
        <position name="position_control2" joint="joint2" kp=".2" kv="0.02" ctrlrange="0 3.1416"/>
        <position name="position_control3" joint="joint3" kp=".2" kv="0.02" ctrlrange="-1.5708 1.5708"/>
    </actuator>
    
    <sensor>
        <jointpos name="joint1_position_sensor" joint="joint1"/>
        <jointpos name="joint2_position_sensor" joint="joint2"/>
        <jointpos name="joint3_position_sensor" joint="joint3"/>
    </sensor>
    
    <keyframe>
        <key name="initial_pose" qpos="0 0 0"/>
        <key name="bent_pose" qpos="0 0 0"/>
    </keyframe>

</mujoco>"""


# load
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
# mujoco.mj_resetDataKeyframe(model, data, 0)
# viewer.launch(model,data)

In [5]:
# 获取链接位置和旋转矩阵的函数
def get_link_pose(model, data, link_name):
    try:
        link_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, link_name)
        position = data.xpos[link_id]
        rotation_matrix = data.xmat[link_id].reshape(3, 3)
        print(f"{link_name} 位置（position）:", position)
        print(f"{link_name} 旋转矩阵（rotation_matrix）:\n", rotation_matrix, "\n")
        return position, rotation_matrix
    except Exception as e:
        print(f"Error accessing {link_name}:", e)
        return None, None

In [6]:
# 定义正向运动学的初始参数
M = np.array([[0, 0, 1, 0.2],
              [0, 1, 0, 0],
              [-1, 0, 0, 0.1],
              [0, 0, 0, 1]])
Blist = np.array([[0, 0, 1, 0, 0, 0],
                  [0, -1, 0, 0.3, 0, -0.2],
                  [1, 0, 0, 0, 0.1, 0]]).T

In [ ]:
mujoco.mj_resetDataKeyframe(model, data, 0)
viewer.launch(model,data)

In [ ]:
mujoco.mj_step(model, data)

# 打印 link1, link2, link3 的位置信息和旋转矩阵
get_link_pose(model, data, "link1")
get_link_pose(model, data, "link2")
get_link_pose(model, data, "link3")

# 获取并打印末端执行器的位置和旋转矩阵
try:
    site_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, "end_effector")
    position = data.site_xpos[site_id]
    rotation_matrix = data.site_xmat[site_id].reshape(3, 3)
    print("末端执行器位置（position）:", position)
    print("末端执行器旋转矩阵（rotation_matrix）:\n", rotation_matrix)
except Exception as e:
    print("Error accessing end_effector:", e)

# 从 MuJoCo 获取关节角度并计算正向运动学
thetalist = data.qpos[:3]
print("\ntheta:", thetalist)
print("\nKinematic position of end-effector in base frame: ")
print(mr.FKinBody(M, Blist, thetalist))

link1 位置（position）: [0.  0.  0.3]
link1 旋转矩阵（rotation_matrix）:
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] 

link2 位置（position）: [0.2 0.  0.3]
link2 旋转矩阵（rotation_matrix）:
 [[ 2.22044605e-16  1.00000000e+00  0.00000000e+00]
 [-2.22044605e-16  0.00000000e+00 -1.00000000e+00]
 [-1.00000000e+00  2.22044605e-16  2.22044605e-16]] 

link3 位置（position）: [ 2.0000000e-01 -4.4408921e-17  1.0000000e-01]
link3 旋转矩阵（rotation_matrix）:
 [[ 2.22044605e-16  1.57009246e-16  1.00000000e+00]
 [-1.57009246e-16  1.00000000e+00 -1.57009246e-16]
 [-1.00000000e+00 -1.57009246e-16  2.22044605e-16]] 

末端执行器位置（position）: [ 2.0000000e-01 -4.4408921e-17  1.0000000e-01]
末端执行器旋转矩阵（rotation_matrix）:
 [[ 2.22044605e-16  1.57009246e-16  1.00000000e+00]
 [-1.57009246e-16  1.00000000e+00 -1.57009246e-16]
 [-1.00000000e+00 -1.57009246e-16  2.22044605e-16]]

theta: [0. 0. 0.]

Kinematic position of end-effector in base frame: 
[[ 0.   0.   1.   0.2]
 [ 0.   1.   0.   0. ]
 [-1.   0.   0.   0.1]
 [ 0.   0.   0.   1. ]]
link1 位置（po